In [ ]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import time
#time

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

from tkinter import *
from tkinter import filedialog
# tk文件导入模块

In [ ]:
def import_table(root):
    """
    return table"选中的表格
    """
    try:
        root.filename = filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.destroy()
            return table
    except Exception as e:
        root.destroy()
        print("导入错误")

def save_model_table1(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该摘要表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该分类表的名称(表1:主题 - 关键词表)(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
        else:
            pass
    except Exception as e:
        pass
    
def save_model_table2(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该摘要表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该分类表的名称(表2：各个文本对应的主题表)(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
        else:
            pass
    except Exception as e:
        pass

In [ ]:
def clean(single_para,stopwords_clean):
    """
    将stopwords和每一段文本取差集，清洗数据
    param single_para:没清洗文本集中的每一段数据
    return list_clean:每一段清洗好的文本
    """
    list_clean = set(single_para).difference(set(stopwords_clean))
    return list_clean

def data_prepare(corpus):
    """
    把数据清理干净
    param text_list:没清洗过的文本列表
    return list_clean:清洗过的文本列表
    """
    table_segments_list = list(map(lambda x:jieba.lcut(x),corpus))
    stopwords = list(map(lambda x:x.strip("\n"),codecs.open('stopwords.txt', 'r', 'utf-8').readlines()))
    stopwords_clean = list(map(lambda x:x.strip("\r"),stopwords))
    list_clean = list(map(clean,table_segments_list,stopwords_clean))
    return list_clean

def tfidf(clean_text):
    """ 
    产生tf-idf的模型
    param clean_text:清洗过的文本列表
    return corpus_tfidf:tf-idf模型
    """
    # 建立词典
    dictionary = corpora.Dictionary(clean_text)

    # 存档词典
    dictionary.save('dict_v1.dict')

    # 建立词袋模型
    corpus = [dictionary.doc2bow(text) for text in clean_text]

    # 建立tf-idf模型
    tfidf = models.TfidfModel(corpus)

    # 将词袋模型，转换为tf-idf模型
    corpus_tfidf = tfidf[corpus]
    return corpus_tfidf


def lda(lda_model,corpus_tfidf,table_original_texts,num_topics):
    """
    使用lda算法进行分类
    param corpus_tfidf:tf-idf模型
    param table_original_texts:原始文本的列表
    param num_topics:分类多少个主题
    return table_keyword_classfication:关键词和关键词的分类表
    return table_originaltext_classfication:原文文档和关键词的分类表
    """
    # 关键词抽取 - 以及关键词属于哪一类
    top_words_per_topic = []
    for t in range(lda_model.num_topics):
        top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 10)])
    table_keyword_classfication = pd.DataFrame(top_words_per_topic)
    #table_keyword_classfication.to_excel("keyword_classification_words.xlsx")
    
    # 原始文章的归类
    list_classification_f = []
    for x in [item for item in lda_model.get_document_topics(corpus_tfidf)]:
        list_possibility = list(map(lambda x:x[1],x))
        list_classification = list(map(lambda x:x[0],x))
        list_possibility_index = list_possibility.index(max(list_possibility))
        result = list_classification[list_possibility_index]
        list_classification_f.append(result)
    table_original_text_classfication = pd.DataFrame([list(table_original_texts),list_classification_f]).T
    return table_keyword_classfication,table_original_text_classfication

def extract_something(pattern,elem):
    try:
        extract_infos = re.search(pattern_judgements,elem).group()
        return extract_infos
    except Exception as e:
        return "None"

In [ ]:
if __name__ == '__main__':
    root = Tk() # 实例化TKinter窗口
    root.withdraw() # 隐藏TKinter窗口
    table_ready_to_eat = import_table(root)
    
    print("----------------------------------------------")
    print("******************表格已经导入******************")
    print("----------------------------------------------")
    print("表格中所有的列:",table_ready_to_eat.columns)
    selected_column = input("请问需要分析哪一列的关键词词频？(输入关键词部分):")
    
    corpus = table_ready_to_eat[selected_column]
    pattern_judgements = re.compile("判决如下:(.*?)。")
    corpus_judgements = list(map(extract_something,len(corpus) * [pattern_judgements],corpus))
    clean_text = data_prepare(corpus_judgements)
    corpus_tfidf = tfidf(clean_text)
    dictionary = corpora.Dictionary(clean_text)
    num_topics = input("请输入需要分类的主题数量:")
    lda_model = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=num_topics, iterations=500)
    
    ### 此处可以得到两个表：
    ### keyword_classfication：为每类主题下的关键词
    ### original_text_classfication：为每句话所归为的类别
    keyword_classfication,original_text_classfication = lda(lda_model,corpus_tfidf,corpus_judgements,num_topics) #产生这两个表
    keyword_classfication.columns = ["属于分类","分类关键词","Weight"]
    original_text_classfication.columns = ["文本","属于分类"]
    save_model_table1(keyword_classfication)
    save_model_table2(original_text_classfication)